In [1]:
!pip install mesa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import display, clear_output
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
import re
import os
from matplotlib.colors import ListedColormap

class Trash(Agent):
    def __init__(self, unique_id, model, amount):
        super().__init__(unique_id, model)
        self.amount = amount

    def step(self):
        pass

class Robot(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.collected_trash = 0
        self.returning = False  # Indica si el robot está regresando a su posición original
        self.origin_pos = None  # Almacena la posición original del robot antes de ir a la papelera

    def move_to(self, destination):
        """
        Move the robot towards the destination using the Manhattan distance.
        """
        current_x, current_y = self.pos
        dest_x, dest_y = destination
        possible_steps = []

        if current_x < dest_x:
            possible_steps.append((current_x + 1, current_y))
        elif current_x > dest_x:
            possible_steps.append((current_x - 1, current_y))

        if current_y < dest_y:
            possible_steps.append((current_x, current_y + 1))
        elif current_y > dest_y:
            possible_steps.append((current_x, current_y - 1))

        possible_steps = [pos for pos in possible_steps if not self.model.grid.out_of_bounds(pos)]
        possible_steps = [pos for pos in possible_steps if not any(isinstance(agent, Robot) for agent in self.model.grid.get_cell_list_contents(pos))]
        possible_steps = [pos for pos in possible_steps if self.model.office_layout[pos[1]][pos[0]] != 'X']

        if possible_steps:
            new_position = self.random.choice(possible_steps)
            self.model.grid.move_agent(self, new_position)

    def step(self):
        if self.collected_trash < 5 and not self.returning:
            # Movimiento aleatorio del robot
            possible_steps = self.model.grid.get_neighborhood(
                self.pos,
                moore=True,
                include_center=False)
            # Filtrar movimientos para evitar salir del grid
            possible_steps = [pos for pos in possible_steps if not self.model.grid.out_of_bounds(pos)]
            # Filtrar movimientos para evitar celdas ocupadas por otros robots
            possible_steps = [pos for pos in possible_steps if not any(isinstance(agent, Robot) for agent in self.model.grid.get_cell_list_contents(pos))]
            # Filtrar movimientos para evitar celdas con "X"
            possible_steps = [pos for pos in possible_steps if self.model.office_layout[pos[1]][pos[0]] != 'X']

            if possible_steps:
                new_position = self.random.choice(possible_steps)
                self.model.grid.move_agent(self, new_position)

                # Recolectar basura si está disponible
                contents = self.model.grid.get_cell_list_contents([self.pos])
                for content in contents:
                    if isinstance(content, Trash):
                        amount = min(5 - self.collected_trash, content.amount)
                        self.collected_trash += amount
                        content.amount -= amount
                        if content.amount == 0:  # Eliminar la basura si la cantidad llega a 0
                            self.model.grid.remove_agent(content)

                # Si el robot tiene 5 unidades de basura, ir a la papelera
                if self.collected_trash == 5:
                    self.returning = True
                    self.origin_pos = self.pos
        else:
            if self.returning:
                # Ir a la papelera para depositar la basura
                if self.pos == self.model.trashcan_pos:
                    self.model.trashcan.amount += self.collected_trash
                    self.collected_trash = 0
                    self.returning = False
                else:
                    self.move_to(self.model.trashcan_pos)
            else:
                # Regresar a la posición original
                if self.pos == self.origin_pos:
                    self.returning = False
                else:
                    self.move_to(self.origin_pos)

class TrashCan(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.amount = 0

    def step(self):
        pass

class OfficeModel(Model):
    def __init__(self, width, height, office_layout):
        super().__init__()
        self.grid = MultiGrid(width, height, False)  # Cambiar a False para desactivar toroidal
        self.schedule = RandomActivation(self)

        self.office_layout = office_layout

        self.trashcan_pos = None
        self.trashcan = None

        # Inicializa la oficina y encuentra la posición de la papelera
        for y in range(height):
            for x in range(width):
                if office_layout[y][x] == 'P':
                    self.trashcan_pos = (x, y)
                    self.trashcan = TrashCan(1, self)
                    self.grid.place_agent(self.trashcan, self.trashcan_pos)
                    self.schedule.add(self.trashcan)

                elif office_layout[y][x] != 'X' and office_layout[y][x] != '0':
                    amount = int(office_layout[y][x])
                    trash = Trash((x, y), self, amount)
                    self.grid.place_agent(trash, (x, y))
                    self.schedule.add(trash)

        # Inicializa los robots
        robot_count = 0
        while robot_count < 5:
            x = self.random.randrange(width)
            y = self.random.randrange(height)
            if office_layout[y][x] == '0':  # Coloca robots solo en celdas vacías
                if not any(isinstance(agent, Robot) for agent in self.grid.get_cell_list_contents((x, y))):
                    robot = Robot(robot_count, self)
                    self.grid.place_agent(robot, (x, y))
                    self.schedule.add(robot)
                    robot_count += 1

    def step(self):
        self.schedule.step()

def visualize_simulation(model, steps=100):
    fig, ax = plt.subplots()

    # Definir el mapa de colores personalizado
    cmap = ListedColormap(['white', 'red', 'green', 'blue', 'orange', 'purple'])

    for step in range(1, steps + 1):
        ax.clear()
        model.step()
        grid = np.full((model.grid.height, model.grid.width), 0)
        for cell in model.grid.coord_iter():
            cell_content, (x, y) = cell
            if len(cell_content) > 0:
                if any(isinstance(agent, Robot) for agent in cell_content):
                    grid[y][x] = 5  # Morado para los robots
                elif any(isinstance(agent, Trash) for agent in cell_content):
                    grid[y][x] = 4  # Naranja para la basura
            else:
                grid[y][x] = 0  # Blanco para celdas vacías

            # Asignar un color a las celdas que contienen "X" en el diseño de la oficina
            if model.office_layout[y][x] == 'X':
                grid[y][x] = 1  # Rojo para las celdas con "X"
            elif model.office_layout[y][x] == 'P':
                grid[y][x] = 3  # Verde claro para la papelera

        # Contador de basura total en la oficina
        total_trash = sum(trash.amount for trash in model.schedule.agents if isinstance(trash, Trash))

        ax.imshow(grid, cmap=cmap, vmin=0, vmax=5)
        # Imprimir el número de paso y la cantidad de basura restante
        ax.text(0.5, 1.01, f"Steps: {step}", transform=ax.transAxes, ha="center")
        ax.text(0.5, 1.05, f"Total Trash: {total_trash}", transform=ax.transAxes, ha="center")
        display(fig)
        clear_output(wait=True)
        time.sleep(0.5)

def read_office_layout(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Eliminar cualquier carácter de nueva línea al final y dividir la primera línea
    n, m = map(int, re.split(r'[,\s]', lines[0].strip()))
    office_layout = [re.split(r'[,\s]', line.strip()) for line in lines[1:]]

    return n, m, office_layout

def run_simulation(file_path, steps=100):
    n, m, office_layout = read_office_layout(file_path)
    model = OfficeModel(m, n, office_layout)
    visualize_simulation(model, steps)

if __name__ == "__main__":
    # Solicitar al usuario que introduzca la ruta del archivo CSV
    input_csv = input("Por favor, introduce la ruta del archivo CSV: ")

    # Comprobar si el archivo existe
    if not os.path.isfile(input_csv):
        print("El archivo no existe. Por favor, comprueba la ruta del archivo e inténtalo de nuevo.")
    else:
        run_simulation(input_csv)


KeyboardInterrupt: Interrupted by user